In [1]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11")

In [4]:
def getQidLabel(label):
    sparql.setQuery("""
    SELECT ?item WHERE {
      ?item rdfs:label \""""+label+"""\"@en
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    result =results["results"]["bindings"][0]["item"]["value"]
    quid = result.split('/')
    return quid[len(quid)-1]
    

In [6]:
def makelabeldict(Qid):
    sparql.setQuery("""
    select distinct ?subject ?subjectLabel ?property ?propertyLabel {
      values (?item) {(wd:"""+Qid+""")}
      ?subject ?predicate ?item .
      ?property wikibase:directClaim ?predicate
      service wikibase:label { bd:serviceParam wikibase:language "en" }
    }Limit 90
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        mydict.append((result["propertyLabel"]["value"], result["subjectLabel"]["value"]))
    return mydict

In [42]:
makelabeldict(getQidLabel('responsory'))

[('instance of', 'Haec dies'),
 ('instance of', 'O Magnum Mysterium'),
 ('instance of', 'Ecce sacerdos magnus'),
 ('subclass of', 'Ad accedentes'),
 ('instance of', 'The righteous perishes'),
 ('instance of', 'Libera me'),
 ('subclass of', 'Tenebrae responsories')]

In [8]:
def research(Qid):
    sparql.setQuery("""
    SELECT ?item ?itemLabel
    WHERE 
    {
      ?item wdt:P31 wd:""" + Qid + """. #on pourais rajouté des obligation de référance à des payes etc
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
      
    }LIMIT 1000
    """)
    #/wdt:P31/wdt:P279
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    mydict = []
    for result in results["results"]["bindings"]:
        # ne prend pas ce ou il y a pas de nom ajouté
        if(result["itemLabel"]["value"][0]!="Q"):
            mydict.append(result["itemLabel"]["value"])
            
    return mydict

In [9]:
listofidqHuman = ['Q22808320', 'Q571', 'Q31629', 'Q188451', 'Q192581', 'Q11424', 'Q11410', 'Q2095', 'Q3624078']


In [10]:
description=[]
for i in range(len(listofidqHuman)):
    description.append(research(listofidqHuman[i]))

In [11]:
for descript in description:
    print(descript)

['David Terry', 'Robert Newton', 'William Meredith', 'Robert Rose', 'William Gibson', 'Richard Thompson', 'John Gillis', 'William Hamilton', 'Ian Hamilton', 'José Martínez', 'James Johnson', 'Alan Ball', 'Thrasydaios', 'Mikhail Biryukov', 'Mark Foster', 'Arthur Mitchell', 'James Rogers', 'George Curzon', 'Francisco Hernández de Córdoba', 'Charles V', 'Johann Strauss', 'Michael Howard', 'Keith Campbell', 'Lukas Bauer', 'David McNally', 'William Jones', 'Gareth Thomas', 'Henry II', 'Jesse Williams', 'Michael Anderson', 'George Bush', 'Chris Nelson', 'John Carroll', "Anna de' Medici", 'Henk Visser', 'Nikos Politis', 'Alan Clark', 'Peter Karageorgevitch', 'Mikko Lehtonen', 'Manuel I', 'Manuel II', 'Margaret of Burgundy', 'Meto Jovanovski', 'Norman Scott', 'Hatice Sultan', 'Michael Clarke', 'Alex Harvey', 'Alexander Campbell', 'Alexander Gray', 'Alexander Smirnov', 'Aleksandr Petrov', 'Alexei Petrov', 'Walter Forster', 'John Chambers', 'Peter Schneider', 'Franz Koch', 'James Dennis', 'John 

In [12]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [13]:
model_path = "./models/"

In [14]:
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =T5ForConditionalGeneration.from_pretrained(model_path, return_dict=True)

In [15]:
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  
    outputs = model.generate(input_ids, max_length=300, temperature=0.1)
    return tokenizer.decode(outputs[0])

In [16]:
import unicodedata

In [17]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [18]:
import gensim

# Load Google's pre-trained Word2Vec model.
word2vecModel = gensim.models.KeyedVectors.load_word2vec_format(model_path+'GoogleNews-vectors-negative300.bin', binary=True) 

In [19]:
#topics contient les sujets pour lesquels il y a des méthodes pour le dialogue
#quand on implémente des nouvelles méthodes il faut rajouter le sujet ici
topics=['music', 'job', 'food', 'game', 'sport', 'book', 'film', 'introduction']

In [20]:
nameofspeaker = ""
askedname = False
class Person:
    
    def __init__(self):
        self.age = random.randint(18, 80)
        self.name = unicodedata.normalize('NFD', random.choice(description[0])).encode('ascii', 'ignore').decode('utf-8')
        self.book = unicodedata.normalize('NFD', random.choice(description[1])).encode('ascii', 'ignore').decode('utf-8')
        self.sport = unicodedata.normalize('NFD', random.choice(description[2])).encode('ascii', 'ignore').decode('utf-8')
        self.music = unicodedata.normalize('NFD', random.choice(description[3])).encode('ascii', 'ignore').decode('utf-8')
        self.job = unicodedata.normalize('NFD', random.choice(description[4])).encode('ascii', 'ignore').decode('utf-8')
        self.film =  unicodedata.normalize('NFD', random.choice(description[5])).encode('ascii', 'ignore').decode('utf-8')
        self.game =  unicodedata.normalize('NFD', random.choice(description[6])).encode('ascii', 'ignore').decode('utf-8')
        self.food =  unicodedata.normalize('NFD', random.choice(description[7])).encode('ascii', 'ignore').decode('utf-8')
        self.country = unicodedata.normalize('NFD', random.choice(description[8])).encode('ascii', 'ignore').decode('utf-8')
        self.passedTopics=[]
        
    def refresh(self):
        self.passedTopics=[]
        
    def generateDescription(self):
        surname=self.name.split(' ')[0]
        string=self.name+' | age | '+str(self.age)+' && '+self.name+' | country | '+self.country
        print(generate(string, model, tokenizer))
        string=surname+' | job | '+self.job
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteBook | '+self.book
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteSport | '+self.sport
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteMusic | '+self.music
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFilm | '+self.film
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteGame | '+self.game
        print(generate(string, model, tokenizer))
        string=surname+' | favouriteFood | '+self.food
        print(generate(string, model, tokenizer))

    def showProperties(self):
        print('name: '+self.name+'\nage: '+str(self.age)+'\ncountry: '+self.country+'\njob: '+self.job+'\nbook: '+self.book+'\nsport: '+self.sport+'\nmusic: '+self.music+'\nfilm: '+self.film+'\ngame: '+self.game+'\nfood: '+self.food+'\n')
        
    def introduce(self):
        text=self.name.split(' ')[0]+':   '

        #Greeting
        rand=random.randint(0,3)
        if rand==0:
            text+='Hello, '
        elif rand==1:
            text+='Hi, '
        elif rand==2:
            text+='Hey, '
        elif rand==3:
            text+='Good day, '
      
        #Name
        rand=random.randint(0,2)
        if rand==0:
            text+="I'm "
        elif rand==1:
            text+="I am "
        elif rand==2:
            text+="my name is "
        text+=self.name+', '

        #Age
        rand=random.randint(0,1)
        if rand==0:
            text+='I am '
        elif rand==1:
            text+="I'm "
        text+=str(self.age)+' years old '

        #Country
        rand=random.randint(0,3)
        if rand==0:
            text+="and I'm from "
        elif rand==1:
            text+="and I am from "
        elif rand==2:
            text+="and I come from "
        elif rand==3:
            text+="and I live in "
        text+=self.country+'.'

        print(text)

    def tellYourFavouriteFood(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite food is '
        elif rand==1:
            text+='I like to eat '
        elif rand==2:
            text+='I love '
        elif rand==3:
            text+='My favourite dish is '
        elif rand==4:
            text+='I really enjoy '
        text+=self.food+'.'
        return text

    def startTopicFood(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFood()+' '
        #Ask the others about their favourite food
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite food?'
        elif rand==1:
            text+='What do you like to eat most?'
        elif rand==2:
            text+='What is your preferred dish?'
        elif rand==3:
            text+='What is your favourite dish?'
        elif rand==4:
            text+='Which food do you enjoy most?'
        print(name+text)
        return text

    def tellYourJob(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='I work as a '
        elif rand==1:
            text+='I am '
        elif rand==2:
            text+="I'm "
        elif rand==3:
            text+='My job is '
        elif rand==4:
            text+='I make a living from working as a '
        text+=self.optimizeJob(self.job)+'.'
        return text
    
    def optimizeJob(self, job):
        split=job.split(' ')
        for i in range(len(split)):
            if split[i]=='management':
                split[i]='manager'
            elif split[i]=='support':
                split[i]='supporter'
            elif split[i]=='therapy':
                split[i]='therapist'
            elif split[i]=='supervision':
                split[i]='supervisor'
            elif split[i]=='scouting':
                split[i]='scout'
        optimizedJob=''
        for word in split:
            optimizedJob+=word+' '
        optimizedJob=optimizedJob[:len(optimizedJob)-1]
        return optimizedJob

    def startTopicJob(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourJob()+' '
        #Ask the others about their job
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your job?'
        elif rand==1:
            text+='What are you doing for a living?'
        elif rand==2:
            text+='What are you working?'
        elif rand==3:
            text+='What do you do?'
        elif rand==4:
            text+='What do you work as?'
        print(name+text)
        return text

    def tellYourFavouriteMusic(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite music is '
        elif rand==1:
            text+='I like listening to '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='My favourite music genre is '
        elif rand==4:
            text+='I really enjoy listening to '
        elif rand==5:
            text+='I usually listen to '
        text+=self.music+'.'
        return text

    def startTopicMusic(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteMusic()+' '
        #Ask the others about their favourite music
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite music?'
        elif rand==1:
            text+='What do you like to listen to?'
        elif rand==2:
            text+='What is your favourite music genre?'
        elif rand==3:
            text+='What music do you like to listen to?'
        elif rand==4:
            text+='Which type of music do you enjoy most?'
        print(name+text)
        return text
        
    def tellYourFavouriteGame(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite game is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        text+=self.game+'.'
        return text

    def startTopicGame(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteGame()+' '
        #Ask the others about their favourite game
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite game?'
        elif rand==1:
            text+='What do you like to play?'
        elif rand==2:
            text+='Which games do you play?'
        elif rand==3:
            text+='Are there games that you like to play?'
        elif rand==4:
            text+='Which game do you enjoy most?'
        print(name+text)
        return text
        
    def tellYourFavouriteSport(self):
        text=''
        rand=random.randint(0,9)
        if rand==0:
            text+='My favourite sport is '
        elif rand==1:
            text+='I like playing '
        elif rand==2:
            text+='I like '
        elif rand==3:
            text+='I like to play '
        elif rand==4:
            text+='I really enjoy playing '
        elif rand==5:
            text+='I often play '
        elif rand==6:
            text+='I like watching '
        elif rand==7:
            text+='I like to watch '
        elif rand==8:
            text+='I\'m a fan of '
        elif rand==9:
            text+='I\'m a '+self.sport+' fan.'
        if not rand==9:
            text+=self.sport+'.'
        return text

    def startTopicSport(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteSport()+' '
        #Ask the others about their favourite sport
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite sport?'
        elif rand==1:
            text+='Are you a sports fan?'
        elif rand==2:
            text+='Which sport are you interested in?'
        elif rand==3:
            text+='What is your preferred sport?'
        elif rand==4:
            text+='Which kind of sport do you like most?'
        print(name+text)
        return text
        
    def tellYourFavouriteFilm(self):
        text=''
        rand=random.randint(0,4)
        if rand==0:
            text+='My favourite film is '
        elif rand==1:
            text+='I really liked the film '
        elif rand==2:
            text+='I really enjoyed watching the film '
        elif rand==3:
            text+='I like films like '
        text+=self.film
        if rand<4:
            text+='.'
        elif rand==4:
            text+=' was the film that I liked most.'
        return text
    
    def startTopicFilm(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteFilm()+' '
        rand=random.randint(0,3)
        if rand==0:
            text+='What is your favourite film?'
        elif rand==1:
            text+='Which film did you like most?'
        elif rand==2:
            text+='Which films do watch?'
        elif rand==3:
            text+='Which film did you enjoy most?'
        print(name+text)
        return text
        
    def tellYourFavouriteBook(self):
        text=''
        rand=random.randint(0,5)
        if rand==0:
            text+='My favourite book is '
        elif rand==1:
            text+='I really like the book '
        elif rand==2:
            text+='I enjoyed reading the book '
        elif rand==3:
            text+='I am currently reading '
        elif rand==4:
            text+='I like the book '
        elif rand==5:
            text+='I find '
        text+=self.book
        if rand<3:
            text+='.'
        elif rand==3:
            text+=' and i really like it.'
        elif rand==4:
            text+=' a lot.'
        elif rand==5:
            text+=' is a pretty good book.'
        return text
    
    def startTopicBook(self):
        name=self.name.split(' ')[0]+':   '
        text=self.tellYourFavouriteBook()+' '
        rand=random.randint(0,4)
        if rand==0:
            text+='What is your favourite book?'
        elif rand==1:
            text+='What do you like to read?'
        elif rand==2:
            text+='Which kind of books do you like?'
        elif rand==3:
            text+='Do you read books?'
        elif rand==4:
            text+='Which book do you like most?'
        print(name+text)
        return text

    def respond(self, message):
        #TODO alles beachten
        global nameofspeaker
        understood=False
        text=self.name.split(' ')[0]+':   '
        phrases = []
        questions = []
        words = ""
        for letter in message:
            if(letter == '.' or letter == '!'):
                phrases.append(words)
                #print(words + " -> is a phrase")
                words = ""
            elif(letter == '?'):
                questions.append(words)
                #print(words + " -> is a question")
                words = ""
            else:    
                words+=letter   
        
        SWITCH_DICT={
            'sport':self.tellYourFavouriteSport,
            'music':self.tellYourFavouriteMusic,
            'job':self.tellYourJob,
            'game':self.tellYourFavouriteGame,
            'food':self.tellYourFavouriteFood,
            'film':self.tellYourFavouriteFilm,
            'book':self.tellYourFavouriteBook
        }
        
        #récupérer les sujets des questions et répondre à une
        topicsquestions = []
        if(len(questions)!= 0):
            for question in questions:
                guessedTopic=self.guessTopic(question)
                if not guessedTopic==None:
                    understood=True
                    if not guessedTopic in self.passedTopics:
                        topicsquestions.append(guessedTopic)
        if(len(topicsquestions)!=0):
            rand=random.randint(0,len(topicsquestions)-1)
            topic=topicsquestions[rand]
            self.passedTopics.append(topic)
            text+=SWITCH_DICT.get(topic)()
            
        #récupérer les sujets des autres phrases et répondre à une
        topicsphrases = []    
        if(len(phrases) != 0):
            for phrase in phrases:
                guessedTopic=self.guessTopic(phrase)
                if not guessedTopic==None:
                    understood=True
                    if not guessedTopic in self.passedTopics:
                        topicsphrases.append(guessedTopic)
        if(len(topicsphrases)!=0):
            rand=random.randint(0,len(topicsphrases)-1)
            topic=topicsphrases[rand]
            if(len(topicsquestions)!=0): 
                text+=' '
            self.passedTopics.append(topic)
            text+=SWITCH_DICT.get(topic)()
            
        #répondre qu'on n'a pas compris la question
        if not understood: 
            rand=random.randint(0,3)
            if rand==0:
                text+='What do you mean by that? '
            elif rand==1:
                text+='Sorry, I don\'t understand. '
            elif rand==2:
                text+='Sorry, I didn\'t get your question. '
            elif rand==3:
                text+='I don\'t know what you are talking about. '
            text+='We can talk about sport, music, work, games, food, films and books.'
            print(text)
            return -1
        #on a compris mais plus rien à dire parce qu'on a déjà parlé sur ce sujet
        if len(topicsphrases)==0 and len(topicsquestions)==0:
            return 0
        print(text)
        
    def guessTopic(self, text):
        bestTopic=None
        bestDistance=None
        words=text.split(' ')
        for word in words:
            #print(word)
            for topic in topics:
                try:
                    distance=word2vecModel.distance(word, topic)
                    #print(distance)
                    if bestTopic==None or distance<bestDistance:
                        bestTopic=topic
                        bestDistance=distance
                except:
                    continue
        return bestTopic
    
    def startTopic(self, doneTopics):
        openTopics=[topic for topic in topics if topic not in doneTopics]
        if len(openTopics)==0:
            return None
        rand=random.randint(0,len(openTopics)-1)
        if openTopics[rand]=='music':
            doneTopics.append('music')
            return self.startTopicMusic()
        elif openTopics[rand]=='job':
            doneTopics.append('job')
            return self.startTopicJob()
        elif openTopics[rand]=='food':
            doneTopics.append('food')
            return self.startTopicFood()
        elif openTopics[rand]=='game':
            doneTopics.append('game')
            return self.startTopicGame()
        elif openTopics[rand]=='sport':
            doneTopics.append('sport')
            return self.startTopicSport()
        elif openTopics[rand]=='film':
            doneTopics.append('film')
            return self.startTopicFilm()
        elif openTopics[rand]=='book':
            doneTopics.append('book')
            return self.startTopicBook()
        else:
            return None

In [21]:
person = Person()
person.respond("my name is Leo . What sport do you like ? i love Sandwiches.")

Humphrey:   I'm a Muay Thai fan. I really enjoy Tauto Pekalongan.


In [22]:
def generatePersons(nPersons):
    persons=[]
    for i in range(nPersons):
        person = Person()
        persons.append(person)
    return persons

In [23]:
def startChat(persons, nTopics):
    doneTopics=[]
    for person in persons: 
        person.introduce()
    for i in range(nTopics):
        personToStart=persons[random.randint(0, len(persons)-1)]
        message=personToStart.startTopic(doneTopics)
        if message==None:
            return
        random.shuffle(persons)
        for person in persons:
            if not person==personToStart:
                person.respond(message)

In [34]:
import time
def startChatInteractive(persons):
    global nameofspeaker
    nameofspeaker=input('Input your name : ')
    for person in persons: 
        person.introduce()
        time.sleep(2)
    while True:
        message=input(nameofspeaker + ': ')
        if message=='bye':
            for person in persons:
                person.refresh()
            return
        if not message=='':
            random.shuffle(persons)
            sleep=True
            rand=random.randint(1, len(persons))
            for person in persons[0:rand]:
                if sleep:
                    time.sleep(2)
                res=person.respond(message)
                if res==-1:
                    break
                elif res==0:
                    sleep=False

In [25]:
def generateDescriptions(persons):
    for person in persons:
        person.generateDescription()
        print('\n')

In [26]:
myPersons=generatePersons(4)

In [27]:
startChat(myPersons, 5)           

Alexei:   Hi, my name is Alexei Petrov, I am 21 years old and I'm from Malawi.
George:   Hey, my name is George Wyndham, I'm 65 years old and I live in Argentina.
Jack:   Hey, my name is Jack Johnson, I am 32 years old and I live in Tunisia.
Frederik:   Hello, I am Frederik av Danmark, I am 71 years old and I live in Kiribati.
Frederik:   I often play Western riding. What is your favourite sport?
Alexei:   I'm a dodgeball fan. I often play Sugaku.
Jack:   I really enjoy playing skeleton. My favourite game is Shout About Movies.
George:   I like 9-man. I often play The Last Resort.
Alexei:   I like to eat Hashasl corek. Which food do you enjoy most?
Jack:   My favourite food is Soto Sokaraja.
George:   My favourite food is Allium sativum.
Frederik:   I like to eat Manisa kebap.
Alexei:   My job is sequence director. What do you do?
Frederik:   I make a living from working as a tonoi.
Jack:   I'm nihon-buyo dancer.
George:   I am baseball commentator.
George:   I usually listen to respon

In [35]:
startChatInteractive(myPersons)

Input your name : stefna
George:   Hey, I am George Wyndham, I'm 65 years old and I come from Argentina.
Frederik:   Good day, my name is Frederik av Danmark, I'm 71 years old and I live in Kiribati.
Alexei:   Hello, I am Alexei Petrov, I am 21 years old and I am from Malawi.
Jack:   Hey, I'm Jack Johnson, I'm 32 years old and I live in Tunisia.
stefna: sport.
Frederik:   I like playing Western riding.
Jack:   I like playing skeleton.
stefna: food. job?
George:   My job is baseball commentator. My favourite food is Allium sativum.
Alexei:   I'm sequence director. My favourite food is Hashasl corek.
Jack:   I'm nihon-buyo dancer. My favourite dish is Soto Sokaraja.
Frederik:   I'm tonoi. I like to eat Manisa kebap.
stefna: bye


In [29]:
generateDescriptions(myPersons)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Alexei Petrov, who is from the country of Malawi, has 21 years.
Alexei performs as a sequence director.
The book The Moki snake Dance is Alexei's favourite book.
The favourite sport of Alexei is dodgeball.
The favourite music of Alexei is Cumbia andina.
The favourite book of Alexei is The Friends.
Sugaku is the favourite games of Alexei.
The favourite food of Alexei is Hashasl corek.


Jack Johnson, who is a member of the country of Tunisia, has a career of 32 years.
The nihon-buyo dancer is the job of Jack.
The favourite book of the Swiss people is the minerals first found in Switzerland.
The skeleton is the favourite sport of the Jack.
The favourite music of the Jack is the music of the Arabst of August, 1907.
St. Trinian's is the favourite for the Jack.


KeyboardInterrupt: 

In [36]:
for person in myPersons:
    person.showProperties()

name: George Wyndham
age: 65
country: Argentina
job: baseball commentator
book: Missal of Prince Novak
sport: 9-man
music: responsory
film: Eugene Onegin
game: The Last Resort
food: Allium sativum

name: Alexei Petrov
age: 21
country: Malawi
job: sequence director
book: The Moki snake dance
sport: dodgeball
music: cumbia andina
film: The Friends
game: Sugaku
food: Hashasl corek

name: Jack Johnson
age: 32
country: Tunisia
job: nihon-buyo dancer
book: Minerals first discovered in Switzerland and minerals named after Swiss individuals
sport: skeleton
music: music of ancient Egypt
film: St Trinian's
game: Shout About Movies
food: Soto Sokaraja

name: Frederik av Danmark
age: 71
country: Kiribati
job: tonoi
book: Libertad bajo palabra
sport: Western riding
music: East Coast hip hop
film: Words in Blue
game: Freespace Fest
food: Manisa kebap



In [38]:
generate('Erzen | country | Albania', model, tokenizer)

'Erzen is in the country of Albania.'